# Construção de agentes autônomos

Esta é uma implementação mínima de um agente autônomo que pode pesquisar no Arxiv por artigos relevantes e resumir as ideias centrais do artigo com base na pergunta de pesquisa do usuário.

Criaremos apenas uma versão altamente abstrata, na qual LangChain faz toda a orquestração sob o capô.



## Instalar pacotes e configurar o ambiente

In [ ]:
!pip install langchain
!pip install arxiv
!pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "" 

- importar pacotes necessários

In [ ]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.agents import Tool
from langchain.chat_models import ChatOpenAI
from langchain.utilities import ArxivAPIWrapper

- Definimos a ferramenta que o modelo de linguagem pode usar: arxiv api (para procurar artigos relevantes), incluindo o nome e as descrições da ferramenta, para que o LLM saiba que pode usar a ferramenta quando necessário

In [ ]:
llm = ChatOpenAI(temperature=0) # Inicialize o LLM para ser usado
arxiv = ArxivAPIWrapper()
arxiv_tool = Tool(
    name="arxiv_search",
    description="Search on arxiv. The tool can search a keyword on arxiv for the top papers. It will return publishing date, title, authors, and summary of the papers.",
    func=arxiv.run
)

tools = [arxiv_tool]

 Em seguida, inicializamos o agente passando as ferramentas e o modelo LLM

In [ ]:
agent_chain = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

- Por fim, fazemos uma pergunta ao agente. O agente usará a estrutura ReAct para concluir a tarefa. Ele primeiro pensa se tem todas as informações necessárias, então decide usar a API do arxiv para procurar artigos relevantes, quando obtém os artigos, resume o conteúdo e os devolve ao usuário.

In [ ]:
agent_chain.run("What is a Large Language Model ?")



> Entering new AgentExecutor chain...
I should use arxiv_search to find papers that define and explain Large Language Models.
Action: arxiv_search
Action Input: "Large Language Model"
Observation: Published: 2022-02-07
Title: Cedille: A large autoregressive French language model
Authors: Martin Müller, Florian Laurent
Summary: Scaling up the size and training of autoregressive language models has
enabled novel ways of solving Natural Language Processing tasks using zero-shot
and few-shot learning. While extreme-scale language models such as GPT-3 offer
multilingual capabilities, zero-shot learning for languages other than English
remain largely unexplored. Here, we introduce Cedille, a large open source
auto-regressive language model, specifically trained for the French language.
Our results show that Cedille outperforms existing French language models and
is competitive with GPT-3 on a range of French zero-shot benchmarks.
Furthermore, we provide an in-depth comparison of the toxici

'Large Language Models are autoregressive language models that have been scaled up in size and training to enable novel ways of solving Natural Language Processing tasks using zero-shot and few-shot learning, but their performance on African languages is largely unknown and they may not be comprehensive models of natural language.'